In [29]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import time
import urllib3
import random
urllib3.disable_warnings()

In [2]:
logindata = {
 'inbox':{"username":"rs31@inbox.ru","password":"ALEX12345"},
 'gmail': {"username":"supermariolanza@gmail.com","password":"Alex12345"}
}

In [4]:
u = logindata['inbox']['username']
p = logindata['inbox']['password']
def get_token(u, p):
    s = requests.Session()
    data = {"username":u,"password":p}
    url = "https://sportme.club/sport/auth/login"
    r = s.post(url, data=data, verify=False)
    request_result = r.json()
    return request_result['token']

In [5]:
u = logindata['inbox']['username']
p = logindata['inbox']['password']
token = get_token(u, p)

In [6]:

def make_rq_url(mydate='2020-08-12', myaddress='52'):
    url = 'https://sportme.club/sport/events/search?zoneId=Europe/Moscow&'
    url += f'objectId={myaddress}&cityId=1'
    url += f"&pageSize=10&date={mydate}%2019:00:00&startTime=07:00"
    url += f"&pageNo=1&rand=0.7210804589143462"
    return url

In [7]:
make_rq_url()

'https://sportme.club/sport/events/search?zoneId=Europe/Moscow&objectId=52&cityId=1&pageSize=10&date=2020-08-12%2019:00:00&startTime=07:00&pageNo=1&rand=0.7210804589143462'

In [8]:
def get_events_info(token,mydate,myaddress='52'):
    myurl = make_rq_url(mydate=mydate, myaddress=myaddress)
    myheaders = {"x-auth-token":token}
    res = requests.get(myurl, headers=myheaders, verify=False)
    return res.json()
    

    
    

In [9]:
# get_events_info(token,mydate='2020-08-11',myaddress='1')

In [13]:
def get_all_events(dates, token, myaddress):
    all_events = []
    for mydate in dates:
        event_json = get_events_info(token, mydate, myaddress)
        eventlist = event_json['data']['newTomorrow']
        new_eventlist = []
        for el in eventlist:
            places = el['places']['free']
            event_id = el['bookId']
            event_start = el['dateBegin']
            new_eventlist.append((event_id,event_start,places))
        all_events.extend(new_eventlist)
        time.sleep(1)
    return all_events

def get_morning_events(all_events):
    morning_events = []
    for el in all_events:
        if el[1].split(' ')[1] == '07:00:00':
            morning_events.append(el[0])
    return morning_events
    

In [14]:
dates = [
    '2020-08-12',
    '2020-08-13' 
]
myaddress='1'
get_morning_events(get_all_events(dates,token,myaddress))

[6114, 6118]

In [15]:
get_all_events(dates,token,'52')

[(6062, '2020-08-13 07:00:00', 0),
 (6068, '2020-08-13 10:00:00', 0),
 (6073, '2020-08-13 12:30:00', 0),
 (6078, '2020-08-13 15:00:00', 0),
 (6079, '2020-08-13 17:30:00', 0),
 (6088, '2020-08-13 20:30:00', 0),
 (6063, '2020-08-14 07:00:00', 0),
 (6067, '2020-08-14 10:00:00', 0),
 (6072, '2020-08-14 12:30:00', 0),
 (6077, '2020-08-14 15:00:00', 0),
 (6081, '2020-08-14 17:30:00', 0),
 (6087, '2020-08-14 20:30:00', 0)]

In [17]:
def get_events_status(events):
    """Получает короткую инфо об ивентах"""
    
    eventids = ','.join([str(x) for x in events])
    url = "https://sportme.club"
    url += f"/sport/events/places?ids={eventids}&rand=0.12051863865376755"
    myheaders = {"x-auth-token":token}
    res = requests.get(url, headers=myheaders, verify=False)
    return res.json()

In [19]:
ei = get_events_status([6060, 6062, 6063])

In [20]:
check_event_freedom(ei)

NameError: name 'check_event_freedom' is not defined

In [21]:
def check_event_freedom(events_info):
    freebookids = []
    for el in events_info['data']:
        if el['free'] > 0:
            freebookids.append(el['bookId'])
    if freebookids:
        return freebookids
    else:
        return None

In [22]:
def check_morning_events(dates,token,myaddress):
    """ Проверяет наличие свободных мест по утрам """
    
    events = get_morning_events(get_all_events(dates,token,myaddress))
    event_info = get_events_status(events)
    result = check_event_freedom(event_info)
    return result
    

In [23]:
check_morning_events(dates,token,'1')

[6114, 6118]

In [24]:
free_bookids = check_morning_events(dates,token,'1')

In [143]:
u = logindata['inbox']['username']
p = logindata['inbox']['password']
token = get_token(u, p)

In [25]:
def check_event_booking(event_id, myheaders):
    url = f'https://sportme.club/sport/events/{event_id}/check?rand=0.4829101962936897'
    res = requests.put(url, headers=myheaders, verify=False)
    book_status = res.json()
    return book_status

In [179]:
check_event_booking(event_id, myheaders)
# {'state': 'OK'} {'message': 'FINISH_BOOKING'}

{'message': 'FINISH_BOOKING'}

In [30]:
def book_event(event_id):
    """Бронирует если ивент еще не забронирован"""
    
    rand = random.random()
    url = f'https://sportme.club/sport/events/{event_id}/reserve?rand={rand}'
    myheaders = {
    "x-auth-token":token,
    'authority': 'sportme.club',
    'method': 'POST',
    'path': f'/sport/events/{event_id}/reserve?rand={rand}',
    'scheme': 'https',
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'clientwidth': '1615',
    'content-length': '0',
    'ip': '185.18.4.94',
    'origin': 'https://sportme.club',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'timezone': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
    }
    checker = check_event_booking(event_id, myheaders)
    if checker.get('state') == 'OK' or checker.get('message') == 'FINISH_BOOKING':
        res = requests.post(url, headers=myheaders, verify=False)
        result = res.json()
        if result['state'] == 'OK':
            print(f'Слот {event_id} забронирован')
            return True
        else:
            print(f'Не получается забронировать {event_id}')
            return None
    else:
        print('checker не пускает ')
        return None
    return 

In [31]:
def unbook_event(event_id):
    rand = random.random()
    url = f'https://sportme.club/sport/events/{event_id}/unreserve?rand={rand}'
    myheaders = {
    "x-auth-token":token,
    'authority': 'sportme.club',
    'method': 'POST',
    'path': f'/sport/events/{event_id}/reserve?rand={rand}',
    'scheme': 'https',
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'clientwidth': '1615',
    'content-length': '0',
    'ip': '185.18.4.94',
    'origin': 'https://sportme.club',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'timezone': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
    }
    
    res = requests.post(url, headers=myheaders, verify=False)
    result = res.json()
    if result['state'] == 'OK':
        print(f'Бронирование {event_id} отменено')
        return True
    else:
        print(f'Не удалось отменить бронь {event_id}')
        return None

In [33]:
book_event(free_bookids[1])

checker не пускает 


In [35]:
unbook_event(free_bookids[1])

Бронирование 6118 отменено


True

In [37]:
booked_events = []
c = 0
while True:
    if c % 100 == 0:
        print(f'Цикл {c} работаем...')
    free_bookids = check_morning_events(dates,token,'52')
    if free_bookids:
        for event_id in free_bookids:
            if event_id not in booked_events:
                b_result = book_event(event_id)
                if b_result:
                    booked_events.append(event_id)
                time.sleep(1)
    time.sleep(30)
    c += 1
    if len(booked_events) > 2 :
        print("Дело сделано")
        break
       

Цикл 0 работаем...
Цикл 100 работаем...
Цикл 200 работаем...
Цикл 300 работаем...
Цикл 400 работаем...
Слот 6062 забронирован
Цикл 500 работаем...
Цикл 600 работаем...
Цикл 700 работаем...
Цикл 800 работаем...
Цикл 900 работаем...
Цикл 1000 работаем...
Цикл 1100 работаем...
Цикл 1200 работаем...
Цикл 1300 работаем...
Цикл 1400 работаем...
Цикл 1500 работаем...
Цикл 1600 работаем...
Цикл 1700 работаем...
Слот 6063 забронирован
Цикл 1800 работаем...
Цикл 1900 работаем...
Цикл 2000 работаем...


KeyboardInterrupt: 